# Decision Tree

## Imports

In [ ]:
import matplotlib
import numpy as np
from sklearn import tree
import pandas as pd

## Import & Pre-process Data

In [ ]:
df=pd.read_csv("heart_data.csv")
df = df.dropna()
df = df.drop('weight', axis=1)
df = df.drop('height', axis=1)

df['bp_mean'] = df[['ap_hi', 'ap_lo']].mean(axis=1)

def categorize_bp(mean_bp):
    if mean_bp > 120:
        return 'High'
    elif mean_bp >= 80:
        return 'Normal'
    else:
        return 'Low'

# Apply the categorization function
df['bp_category'] = df['bp_mean'].apply(categorize_bp)

df.head()

## Fit Data

## Display Data